# Nested Sampling Algorithm

- Jeffrey's prior for $D_0$
- Uniform Priors for $\alpha$, $f$ and $\sigma^2_{mn}$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import scipy.stats as stats
import warnings

In [2]:
def jeff(x,magmin, magmax):
    return magmin*exp(u*log(magmax/magmin))
def uni(x, magmin, magmax):
    return 1/(magmax, magmin)


## likelihood calc function

In [3]:
@jit
def noisylikelihoodcalc(posn, DT, D, ALPHA, F =0 , varmn = 0):
    # time series
    n = len(posn)

    #init value
    logl = 0
    k_BT =1
    

    # Intial values of the variance mean both noisy and clean
    mob = D * (np.abs(posn[0]) ** ALPHA)
    std_dev_noise = 2 * mob * DT + 2 * varmn
    noisy_mean = posn[0] + (mob * F + k_BT * (ALPHA * D * ((np.abs(posn[0]) ** (ALPHA - 1)) * (np.sign(posn[0]))))) * DT
    
    # log likelihood at 1st step
    logl = logl - ((posn[1] - noisy_mean) ** 2) / (2 * std_dev_noise) - np.log(np.sqrt(2 * np.pi * std_dev_noise))
    
    for i in range(2,n):
        # Mobility at the step
        mob = D * np.abs(posn[i - 1]) ** ALPHA

        # clean variance at the jth step
        std_dev = 2 * mob * DT

        # clean mean position at the jth time step
        mean_dist = posn[i - 1] + (mob * F + ALPHA * D * (np.abs(posn[i - 1]) ** (ALPHA - 1)) * (np.sign(posn[i - 1]))) * DT

        # noisy mean position at the jth step
        noisy_mean = mean_dist - varmn/(std_dev_noise) * (posn[i-1] - noisy_mean)

        # noisy variance at the jth posn
        std_dev_noise = std_dev + varmn * (2 - varmn / std_dev_noise)

        # calculation of log likelihood
        logl = logl - ((posn[i] - noisy_mean) ** 2) / (2 * std_dev_noise) - 0.5 * np.log(2 * np.pi * std_dev_noise)
    
    return logl

## Model for calculating Evidence and with consider Model parameters

Instead of defining different functions for different models we could use the theta_max and theta_min vectors to define the parameters properly.

In [4]:
@jit
def Evidence(data, theta, npoints, tol):
    logZ = 0
    walker = walkergen(npoints, theta_max, theta_min)
    for i in range(0,npoints):
        ll[i] = noisylikelihoodcalc(p_data, 1, walker[i][0], walker[i][1], walker[i][2], walker[i][3])
    
    

In [5]:
@jit
def walkergen(n, tmax, tmin):
    x = np.zeros((n,4))
    for i in range(0,n):
        for j in range(0,4):
            x[i][j] = np.random.uniform(tmin[j],tmax[j])
    return x

In [6]:
# @jit 
def logsum(a, b):
    return max(a,b) + np.log(1+ np.exp(-np.abs(a-b)))

## $\theta_{max}$ and $\theta_{min}$ vectors

we use this to define the priors

In [7]:
D_min = 10**(-4)
D_max = 10**2
# D_min = 0.
# D_max = 1.


f_min = -1.
f_max = 1.

alpha_min = -2.
alpha_max = 2.

varmn_min = 0.
varmn_max = 0.

theta_max_pc = np.array([D_max, alpha_max, f_max, varmn_max])
theta_min_pc = np.array([D_min, alpha_min, f_min, varmn_min])

theta_max_fc = np.array([D_max, alpha_max, 0, varmn_max])
theta_min_fc = np.array([D_min, alpha_min, 0, varmn_min])


In [11]:
p_data = np.array([50,46.743,46.751,40.915,45.066,50.56,44.448,40.518,40.145,39.551,42.887,40.218,44.399,38.416,36.626,35.867,30.252,28.843,26.412,27.815,28.166,32.899,34.738,33.237,24.448,22.771,19.285,24.966,23.912,23.839,23.83,22.253,15.661,20.156,20.166,16.9,15.495,18.734,18.234,17.823,16.052,11.424,9.4223,7.3746,7.2549,7.2847,6.7366,7.7324,6.8572,7.034,5.3564,3.7969,3.4073,4.3653,4.56,4.8884,3.8021,2.9259,3.5701,4.0879,2.3471,0.95298,1.7231,1.7968,1.753,2.1651,2.1354,2.404,2.9188,1.1974,1.3358,1.428,1.8476,2.1991,1.1013,1.3227,0.71634,0.68376,0.89795,1.6167,2.0934,2.4119,1.0693,1.2777,2.2396,0.30283,0.64551,-0.44319,-0.79814,-0.82102,-2.0478,-1.6834,-1.8176,-1.3092,-1.2355,-1.8231,-1.8543,-2.1829,-4.2785,-4.2926])

# MCMC walker

Tried my hand at coding a MCMC walker using the Metropolis Hastings algo. The proposal function is a standard gaussian. To make it compatible for different models use can use different step sizes and then set the step sizes for the non fixed parameters to be the same and step size for the fixed parameter to be 0. So the walker is also constrained to walk over a surface that has fixed parameter of the model to be fixed. Setting the proposal function like this means we can define an easier code snippet without having to define multiple methods for different models and then it would be easier to define.

We could also go an easier way and then generate the walkers only in the constrained surface

Have to try both, but have a feeling that the secind method will be more efficient.

In [12]:
# @jit
def MCMCwalker(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,4):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [13]:
# @jit
def MCMCwalkerpc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in range(0,3):
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

In [14]:
# @jit
def MCMCwalkerfc(x, DT, walker, Lstar, N, step = 0):
    
    if step == 0:
        step = 1

    # assigning parameter values for the walker 
    D = walker[0]
    a = walker[1]
    f = walker[2]
    mn = walker[3]
    
    R = 0

    # intializing the walker distribution
    walker_new = np.zeros((N,4))

    # init proposal function
    # proposal for new points in parameter space
    prop = np.zeros(4)

    #rejection count to determine new steps
    rejcount = 0 

    #initial walker value is the intial walker point
    walker_new[0] = walker 

    #counter variable
    i = 1
    
    while i<N:
        
        #setting new values using the proposal function
        for j in [0,1]:
            prop[j] = walker_new[i-1][j] + np.random.normal(0,step)

        # log likelihood of the walker
        llwalker = noisylikelihoodcalc(x, DT, *prop)

        #moving around prior landscape
        if llwalker >= Lstar:
            walker_new[i] = prop
            Lstar = llwalker
        else:
            walker_new[i] = walker_new[i-1]
            rejcount += 1
        i+=1

    # Rejection Ratio
    R = rejcount/N

    #new step
    step = min(step * np.exp( 0.5 - R), 1)
    
    return step, walker_new[-1]         

# Nested Sampling Code starts here

In [23]:
# steps for the random walkers in the MCMC algo
Nsteps = 10000

# no of walkers considered
npoints = 100

# init likelihood of pull, clean and free, clean models
llpc = np.zeros(npoints)
llfc = np.zeros(npoints)

# walkers pull clean and walker free clean
walkers_pc = np.array(walkergen(npoints, theta_max_pc,theta_min_pc))
walkers_fc = np.array(walkergen(npoints, theta_max_fc,theta_min_fc))

print(walkers_fc)
print(walkers_pc)

# likelihood  of Mpc and Mfc
for i in range(0,npoints):
    llpc[i] = noisylikelihoodcalc(p_data, 1, *walkers_pc[i])
    llfc[i] = noisylikelihoodcalc(p_data, 1, *walkers_fc[i])


# wt = 1/(npoints + 1)
logwt = -np.log(npoints + 1)

#Zpc is Z pull clean
#Zfc is Z free clean

# these are the remainders in the jth iteration
logZpcrem = np.log(0)
logZfcrem = np.log(0)

# these are the Log Evidences for the respective models
logZpc = np.log(0)
logZfc = np.log(0)


logp_pc = min(llpc) + logwt
logp_fc = min(llfc) + logwt

logZpc = logsum(logZpc,logp_pc)
logZfc = logsum(logZfc,logp_fc)


for i in range(0, npoints):
    logZfcrem = logsum(llfc[i], logZfcrem)
    logZpcrem = logsum(llpc[i], logZpcrem)

logZfcrem = logZfcrem + logwt
logZpcrem = logZpcrem + logwt

logZpc,min(llpc),logZfc,min(llfc)

[[ 8.90461825e+01 -4.81051345e-01  0.00000000e+00  0.00000000e+00]
 [ 1.61192469e+01  1.41941115e+00  0.00000000e+00  0.00000000e+00]
 [ 2.08049808e+00  1.15308013e+00  0.00000000e+00  0.00000000e+00]
 [ 8.86925493e+01 -1.50204147e-01  0.00000000e+00  0.00000000e+00]
 [ 6.89478695e+01 -1.00655902e+00  0.00000000e+00  0.00000000e+00]
 [ 4.78589100e+00  7.55657053e-01  0.00000000e+00  0.00000000e+00]
 [ 2.40305737e+01 -3.49808684e-01  0.00000000e+00  0.00000000e+00]
 [ 6.41031677e+01  1.02415303e+00  0.00000000e+00  0.00000000e+00]
 [ 8.98174847e+01 -5.57873979e-01  0.00000000e+00  0.00000000e+00]
 [ 1.25653825e+01 -3.19314467e-01  0.00000000e+00  0.00000000e+00]
 [ 6.88933404e+01  1.31287514e+00  0.00000000e+00  0.00000000e+00]
 [ 2.27446872e+01 -1.15087622e+00  0.00000000e+00  0.00000000e+00]
 [ 3.03559139e+01 -1.88302997e+00  0.00000000e+00  0.00000000e+00]
 [ 8.31205844e+00  6.97398106e-02  0.00000000e+00  0.00000000e+00]
 [ 1.40413006e+01  9.42626311e-01  0.00000000e+00  0.00000000e

/tmp/ipykernel_37612/1657887835.py:31: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)
/tmp/ipykernel_37612/1657887835.py:32: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)
/tmp/ipykernel_37612/1657887835.py:35: RuntimeWarning: divide by zero encountered in log
  logZpc = np.log(0)
/tmp/ipykernel_37612/1657887835.py:36: RuntimeWarning: divide by zero encountered in log
  logZfc = np.log(0)


(-541961.8864585394,
 -541957.2713380225,
 -51631.52240145746,
 -51626.90728094062)

In [24]:
# initial stepsize
step = 0

# int index of the loop
j=1

# tolerance value for stopping condition
tol = np.log(10**(-4))

# lowest likelihood in each iteration
lstarpc = []

# log weight
logwt = -np.log(npoints + 1)

# posterior avergae and posterior samples
lpostavgpc = np.zeros(4)
lpostpc = np.zeros(4)

#stopping ratio calculator
Rpc = logZpcrem - logZpc

if Rpc < tol:
    print("term1 stop")
    print(logZpc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglpc = [noisylikelihoodcalc(p_data, 1, *walkers_pc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarpc.append(min(loglpc))

    # walker index with minimum loglikelihood
    ppc = np.argmin(loglpc)

    # previous iterations log Evidence
    prevlogZpc = logZpc
    
    # log Z finding
    logZpc = logsum(logZpc, (lstarpc[-1] + logwt))

    # posterior sample
    lpostpc = (walkers_pc[ppc])*np.exp(lstarpc[-1])*np.exp(logwt)

    # posterior summation after each iteration
    lpostavgpc = lpostavgpc + lpostpc

    print("posterior");
    print(lpostavgpc);
    
    print("Evidence");
    print(logZpc);

    # init Zremaining for the jth iteration
    logZpcrem = np.log(0)

    # calculate Zremaining for the jth iteration
    for i in range(0, npoints):
        logZpcrem = logsum(loglpc[i], logZpcrem)
        
    logZpcrem = logZpcrem + logwt

    print( "Zrem =  "+str(logZpcrem));
    print("lowest likelihood = "+str(min(loglpc)))

    # ratio that determines the stopping ratio
    Rpc = logZpcrem - logZpc
    
    print("ratio")
    print(Rpc)

    #break if stopping ratio is less than tolerance
    if Rpc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_pc[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerpc(p_data, 1, thetawalker, lstarpc[-1] , Nsteps, step)

    # step modulation in MCMC walker
    step = trace[0]

    # replacing the worst walker with the copied walker that undergoes a random walk
    walkers_pc[ppc] = trace[1]

# final evidence calulcation
LogfinalZpc = logsum(prevlogZpc, logZpcrem)

LogfinalZpc,lpostavgpc

posterior
[0. 0. 0. 0.]
Evidence
-541961.1982741482
Zrem =  -299.0142941525617
lowest likelihood = -541957.2713380225
ratio
541662.1839799955
posterior
[0. 0. 0. 0.]
Evidence
-426578.6633975862
Zrem =  -184.817836410195
lowest likelihood = -426574.02837640763
ratio
426393.845561176
posterior
[0. 0. 0. 0.]
Evidence
-241591.15930258067
Zrem =  -184.8277867410482
lowest likelihood = -241586.51433107126
ratio
241406.3315158396


/tmp/ipykernel_37612/1357180594.py:59: RuntimeWarning: divide by zero encountered in log
  logZpcrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-210711.63161952785
Zrem =  -184.83773707190136
lowest likelihood = -210706.97669768758
ratio
210526.79388245594
posterior
[0. 0. 0. 0.]
Evidence
-142486.05601514
Zrem =  -184.84768740275453
lowest likelihood = -142481.3911429689
ratio
142301.20832773726
posterior
[0. 0. 0. 0.]
Evidence
-132433.834647661
Zrem =  -184.8576377336077
lowest likelihood = -132429.15982515903
ratio
132248.9770099274
posterior
[0. 0. 0. 0.]
Evidence
-102499.39794171171
Zrem =  -183.73827471081125
lowest likelihood = -102494.7131688789
ratio
102315.6596670009
posterior
[0. 0. 0. 0.]
Evidence
-65943.13624703122
Zrem =  -183.74822504166443
lowest likelihood = -65938.44152386754
ratio
65759.38802198955
posterior
[0. 0. 0. 0.]
Evidence
-52544.70523860505
Zrem =  -183.7581753725176
lowest likelihood = -52540.00056511053
ratio
52360.94706323253
posterior
[0. 0. 0. 0.]
Evidence
-43759.80939534386
Zrem =  -183.76812570337077
lowest likelihood = -43755.09477151849
ratio
43576.0412696404

(-183.28043588084364,
 array([ 4.57490468e-81,  2.46578852e-80, -6.89648555e-81,  0.00000000e+00]))

In [25]:
# init posterior avg at the jmax iteration
lapc = np.zeros(4)

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lapc += walkers_pc[i] * np.exp(llpc[i])
lapc *= np.exp(logwt)

# final posterior average
postfinpc = lpostavgpc+lapc

# dividing by the evidence of the model to find the posterior
normpostfinpc = postfinpc/np.exp(LogfinalZpc)

normpostfinpc,LogfinalZpc

(array([ 0.1811607 ,  0.97642248, -0.2730925 ,  0.        ]),
 -183.28043588084364)

In [26]:
#initial step size
step = 0

# init index
j=1

#tolerance for stopping condition
tol = np.log(10**(-4))

# lowest likelihood for the jth iteration
lstarfc = []

# log wth value for prior mass
logwt = -np.log(npoints + 1)

# posterior average and posterior samples
lpostavgfc = np.zeros(4)
lpostfc = np.zeros(4)

# stopping ratio
Rfc = logZfcrem - logZfc

# stopping condition
if Rfc< tol:
    print(logZfc)

while True:
    # log width contraction
    logwt = logwt + np.log(npoints/(npoints+1))

    # log likelihood calculation
    loglfc = [noisylikelihoodcalc(p_data, 1, *walkers_fc[i]) for i in range(0,npoints)]

    # lowest likelihood of all the walkers
    lstarfc.append(min(loglfc))

    pfc = np.argmin(loglfc)

    # storing the evidence of the previous iteration
    prevlogZfc = logZfc
    
    # log Z finding
    logZfc = logsum(logZfc, (lstarfc[-1] + logwt))

    # posterior samples
    lpostfc = (walkers_fc[pfc])*np.exp(lstarfc[-1])*np.exp(logwt)
    
    # posterior average
    lpostavgfc = lpostavgfc + lpostfc

    print("posterior");
    print(lpostavgfc);
    
    print("Evidence");
    print(logZfc);

    # init Z remaining of the jth iteration
    logZfcrem = np.log(0)

    # calculating Z remaning of the jth iteration
    for i in range(0, npoints):
        logZfcrem = logsum(loglfc[i], logZfcrem)
        
    logZfcrem = logZfcrem + logwt

    print( "Zrem =  "+str(logZfcrem));
    print("lowest likelihood = "+str(min(loglfc)))

    # ratio that determines the stopping ratio
    Rfc = logZfcrem - logZfc
    
    print("ratio")
    print(Rfc)
    
    if Rfc < tol:
        break
        
    # randomly select a walker to copy and random walk
    newwalkernumber = np.random.randint(0,npoints)

    # parameters of the randomly selected walker
    thetawalker = walkers_fc[newwalkernumber]

    # walker replacement  
    trace = MCMCwalkerfc(p_data, 1, thetawalker, lstarfc[-1] , Nsteps, step)

    # step modulation for the random walk
    step = trace[0]

    # replacement of the worst walker with walker performing random walk
    walkers_fc[pfc] = trace[1]

#final Z calculation
LogfinalZfc = logsum(prevlogZfc, logZfcrem)

LogfinalZfc,lpostavgfc

posterior
[0. 0. 0. 0.]
Evidence
-51630.83421706624
Zrem =  -244.70567207050703
lowest likelihood = -51626.90728094062
ratio
51386.128544995736
posterior
[0. 0. 0. 0.]
Evidence
-9468.027573773785
Zrem =  -188.52680211880676
lowest likelihood = -9463.392552595236
ratio
9279.500771654977
posterior
[0. 0. 0. 0.]
Evidence
-9159.642234489987
Zrem =  -187.83953388506038
lowest likelihood = -9154.997262980587
ratio
8971.802700604927
posterior
[0. 0. 0. 0.]
Evidence
-9072.818800745576
Zrem =  -187.44011109486297
lowest likelihood = -9068.163878905321
ratio
8885.378689650712


/tmp/ipykernel_37612/3209253099.py:58: RuntimeWarning: divide by zero encountered in log
  logZfcrem = np.log(0)


posterior
[0. 0. 0. 0.]
Evidence
-8772.987343414155
Zrem =  -187.16151583238283
lowest likelihood = -8768.32247124305
ratio
8585.825827581773
posterior
[0. 0. 0. 0.]
Evidence
-7953.783496806616
Zrem =  -187.171466163236
lowest likelihood = -7949.108674304656
ratio
7766.61203064338
posterior
[0. 0. 0. 0.]
Evidence
-7880.411352524579
Zrem =  -186.95599723696745
lowest likelihood = -7875.726579691766
ratio
7693.455355287612
posterior
[0. 0. 0. 0.]
Evidence
-7876.784144288937
Zrem =  -186.7821088659394
lowest likelihood = -7872.116371357011
ratio
7690.002035422997
posterior
[0. 0. 0. 0.]
Evidence
-6995.2637916893345
Zrem =  -186.63678314359638
lowest likelihood = -6990.559118194815
ratio
6808.627008545738
posterior
[0. 0. 0. 0.]
Evidence
-6886.683567688956
Zrem =  -186.51235223286
lowest likelihood = -6881.968943863583
ratio
6700.1712154560955
posterior
[0. 0. 0. 0.]
Evidence
-6748.481399963359
Zrem =  -186.40385657979076
lowest likelihood = -6743.756825807132
ratio
6562.077543383568
poste

(-186.00298691457104,
 array([2.75132317e-82, 1.71823849e-81, 0.00000000e+00, 0.00000000e+00]))

In [27]:
# init posterior avg at the jmax iteration
lafc = np.zeros(4)

# calculation of posterior avg at the jmax iteration
for i in range(0,npoints):
    lafc += walkers_fc[i] * np.exp(llfc[i])
lafc *= np.exp(logwt)

#final posterior calculation
postfinfc = lpostavgfc+lafc

# dividing the posterior by the evidence
normpostfinfc = postfinfc/np.exp(LogfinalZfc)

normpostfinfc,LogfinalZfc

(array([0.16581066, 1.03550995, 0.        , 0.        ]), -186.00298691457104)

In [28]:
# seeing the difference of the Evidence
LogfinalZpc - LogfinalZfc

2.722551033727399

In [29]:
# Model probability of the models considered
np.exp(LogfinalZpc)/(np.exp(LogfinalZfc)+np.exp(LogfinalZpc))

0.938344287113444

In [31]:
# ratio of evidences considered
np.exp(LogfinalZpc)/(np.exp(LogfinalZfc))

15.219097196069379

# irrelevant shit to test my moronic code

In [81]:
N=500000
a = walkergen(1,theta_max,theta_min)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
a,lla

(array([ 6.97170044,  1.88013819, -0.38935263,  0.87740753]),
 -6208.648723079805)

In [83]:
a = walkergen(1,theta_max,theta_min)[0]
lla = noisylikelihoodcalc(p_data, 1, *a)
print("init_Walker = " + str(a))
print("init likelihood = " +str(lla))
t = MCMCwalker(p_data, 1, a, lla, N, step)
step = t[0]
fp = t[2]
llf = noisylikelihoodcalc(p_data, 1, fp[0], fp[1], fp[2], fp[3])
fp
t[1]
llf,fp

init_Walker = [4.04643391 1.3696873  0.68423192 0.28069053]
init likelihood = -3145.2794342032416


(-179.1692711507324,
 array([ 0.14287875,  1.05337414, -0.27326816,  0.02651726]))

In [19]:
llt = noisylikelihoodcalc(p_data, 1, 0.2, 1,-0.3,0)
llt

-180.397045194095